In [1]:
from flask import Flask,request,render_template
import pickle
import numpy as np
import flask
import tensorflow as tf
import cv2
 

In [2]:
app = Flask(__name__) 

def sendResponse(responseObj):
    response = flask.jsonify(responseObj)
    response.headers.add('Access-Control-Allow-Origin', '*')
    response.headers.add('Access-Control-Allow-Methods', 'GET')
    response.headers.add('Access-Control-Allow-Headers', 'accept,content-type,Origin,X-Requested-With,Content-Type,access_token,Accept,Authorization,source')
    response.headers.add('Access-Control-Allow-Credentials', True)
    return response

model_image=tf.keras.models.load_model('Covid_19_detection.h5')

APP_ROOT = os.path.dirname(os.path.abspath(__file__))
app.config['IMAGE_UPLOADS'] = os.path.join(APP_ROOT, 'static')

@app.route('/upload')  
def upload():  
    return render_template("upload.html") 

@app.route('/uploader', methods = ['GET', 'POST'])  
def uploader():  
    if request.method == 'POST':  
        file_path = os.path.join(app.config["IMAGE_UPLOADS"], filename)
        file = request.files['file']
        #print(file)
        image=cv2.imread(file.filename)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (224, 224))
        images_final=np.array(image)/255
        pred_1=model.predict(np.array([images_final]))
        
        return sendResponse(str(np.round(pred_1[0][0])))
        
        
        
        #return render_template("success.html", name = f.filename) 
if __name__=='__main__':
    app.run(host='0.0.0.0',port=5000)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


E1018 01:47:30.580323 139827131434752 app.py:1761] Exception on /uploader [POST]
Traceback (most recent call last):
  File "/home/abhijit/.local/lib/python3.6/site-packages/flask/app.py", line 2292, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/abhijit/.local/lib/python3.6/site-packages/flask/app.py", line 1815, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/abhijit/.local/lib/python3.6/site-packages/flask/app.py", line 1718, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/home/abhijit/.local/lib/python3.6/site-packages/flask/_compat.py", line 35, in reraise
    raise value
  File "/home/abhijit/.local/lib/python3.6/site-packages/flask/app.py", line 1813, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/abhijit/.local/lib/python3.6/site-packages/flask/app.py", line 1799, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-2-596392c56